In [5]:
import pandas as pd
import numpy as np
import praw

In [7]:
# hide my api access stuff
auth = open("secret", "r").read().split('|')
CLIENT_ID = auth[0]
SECRET_KEY = auth[1]

In [38]:
# make a PRAW reddit instance
reddit = praw.Reddit (
    client_id=CLIENT_ID,
    client_secret=SECRET_KEY,
    user_agent="uhhh idk",
)

USER_ONE = 'jmct' #professor
USER_TWO = 'ariqbailey'

In [39]:
# get users comments
LIMIT = 20
user_one_comments = reddit.redditor(USER_ONE).comments.new(limit=LIMIT)
one_comment_df = pd.DataFrame(columns=['body', 'body_length', 'subreddit', 'created_utc'])

i = 0
for comment in user_one_comments:
    # print('-'*20)
    # print(comment.subreddit)
    # print(comment.body)
    # print(comment.created_utc)
    one_comment_df.loc[i] = [comment.body, len(comment.body), comment.subreddit, comment.created_utc]
    i += 1

--------------------
UMD
Depends what you mean by Professor.

Technically, no. But most faculty research positions will require a PhD or a _very significant_ publication record (which is harder without a PhD)

If by ‘Professor’ you mean ‘person who teaches at the university level’ then no, there are non-research positions that are more lax about needing a PhD.

Some students call all their lecturers ‘Professor’ even though that’s _technically_ a research title at UMD. Some schools have “teaching Professors”, but most just call them lecturers.
1616204439.0
--------------------
UMD
I find it very demoralizing and, worse, difficult to gauge how the material is landing.

I don’t think we have it nearly as difficult as the students though. It’s just a mess all around.
1614618300.0
--------------------
musictheory
I actually thought the point was more interesting than that.

Without perfect pitch you _have_ to learn ear training skills that you can use _even after you get old_. Whereas if yo

In [40]:
one_comment_df.head()

,body,body_length,subreddit,created_utc
0,Depends what you mean by Professor.\n\nTechnic...,523,UMD,1.616204e+09
1,"I find it very demoralizing and, worse, diffic...",186,UMD,1.614618e+09
2,I actually thought the point was more interest...,368,musictheory,1.612901e+09
3,"It all seems relatively benign, which just rai...",72,law,1.611119e+09
4,I’ve always found mutt to be clunky with multi...,199,linux,1.610816e+09


In [48]:
# get users posts
user_one_posts = reddit.redditor(USER_ONE).submissions.new(limit=LIMIT)
one_post_df = pd.DataFrame(columns=['title', 'body', 'body_length', 'subreddit', 'created_utc'])

i = 0
for post in user_one_posts:
    one_post_df.at[i] = [post.title, post.selftext, len(post.selftext), post.subreddit, post.created_utc]
    i += 1

In [49]:
one_post_df.head()

,title,body,body_length,subreddit,created_utc
0,Trump issues new Executive Order relating to t...,,0,law,1.61112e+09
1,"Any insight into Joe Pass's approach to ""Let i...","Hey all,\n\nI've been transcribing Joe Pass's ...",1131,jazzguitar,1.60707e+09
2,What About the Natural Numbers? PWLConf 2019 [...,,0,programming,1.56933e+09
3,Hoping for constructive criticism and advice o...,,0,apexlegends,1.5588e+09
4,ICFP 2018 Accepted (or conditionally accepted)...,,0,haskell,1.52657e+09
